### 构建一个简单的烷烃
- 演示烷烃聚合物的构造

In [4]:
# 1. 建立单体
# 单体的这种构型不是一种特别现实的构象。
# 人们可以使用这种单体来构建聚合物，然后应用能量最小化方案，
import mbuild as mb

class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        # Add carbon
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')

        # Add hydrogens
        self.add(mb.Particle(name='H', pos=[-0.109, 0, 0.0]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0.109, 0, 0.0]), label='HC[$]')

        # Add bonds between the atoms
        self.add_bond((self['C'][0], self['HC'][0]))
        self.add_bond((self['C'][0], self['HC'][1]))

        # Add ports anchored to the carbon
        self.add(mb.Port(anchor=self[0]), label='up')
        self.add(mb.Port(anchor=self[0]), label='down')

        # Move the ports approximately half a C-C bond length away from the carbon
        self['up'].translate([0, -0.154/2, 0])
        self['down'].translate([0, 0.154/2, 0])

monomer = CH2()
monomer.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
import numpy as np
# 或者，正如我们将在这里演示的那样，
# 我们可以使用mBuild的旋转命令来提供更现实的起点。
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        # Add carbon
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')

        # Add hydrogens
        self.add(mb.Particle(name='H', pos=[-0.109, 0, 0.0]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0.109, 0, 0.0]), label='HC[$]')

        # Rotate the hydrogens
        theta = 0.5 * (180 - 109.5) * np.pi / 180
        #mb.rotate(self['HC'][0], theta, around=[0, 1, 0])
        #mb.rotate(self['HC'][1], -theta, around=[0, 1, 0])
        self['HC'][0].rotate( theta, around=[0, 1, 0])
        self['HC'][1].rotate(-theta, around=[0, 1, 0])

        # Add bonds between the atoms
        self.add_bond((self['C'][0], self['HC'][0]))
        self.add_bond((self['C'][0], self['HC'][1]))

        # Add the ports and appropriately rotate them
        self.add(mb.Port(anchor=self[0]), label='up')
        self['up'].translate([0, -0.154/2, 0])
        self['up'].rotate(theta, around=[1, 0, 0])

        self.add(mb.Port(anchor=self[0]), label='down')
        self['down'].translate([0, 0.154/2, 0])
        self['down'].rotate(-theta, around=[1, 0, 0])

monomer = CH2()
monomer.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
# 2. 定义聚合类
# 有了基本的单体结构，我们现在可以通过连接端口来构建聚合物。
# 在这里，我们首先将CH2类的一个实例实例化为1ast_monomer，然后使用clone函数进行复制。
# force_overlap()函数用于连接current_monomer的'up'端口和last_mononer的'down'端口。

class AlkanePolymer(mb.Compound):
    def __init__(self):
        super(AlkanePolymer, self).__init__()
        last_monomer = CH2()
        self.add(last_monomer)
        for i in range(3):
            current_monomer = CH2()
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer = current_monomer

polymer = AlkanePolymer()
polymer.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# 这个结构的可视化显示了一个问题;聚合物自己卷曲起来。
# 这是因为端口不仅定义了空间中的位置，而且还定义了方向。
# 这可以通过围绕y轴旋转向下端口180°来简单地固定。

# 我们还可以在for循环和init中添加一个变量chain_length，
# 这将允许在类实例化时调整聚合物的长度。

class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
         # Add carbons and hydrogens
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')
        self.add(mb.Particle(name='H', pos=[-0.109, 0, 0.0]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0.109, 0, 0.0]), label='HC[$]')

        # rotate hydrogens
        theta = 0.5 * (180 - 109.5) * np.pi / 180
        self['HC'][0].rotate(theta, around=[0, 1, 0])
        self['HC'][1].rotate(-theta, around=[0, 1, 0])

        # Add bonds between the atoms
        self.add_bond((self['C'][0], self['HC'][0]))
        self.add_bond((self['C'][0], self['HC'][1]))

        # Add ports
        self.add(mb.Port(anchor=self[0]), label='up')
        self['up'].translate([0, -0.154/2, 0])
        self['up'].rotate(theta, around=[1, 0, 0])

        self.add(mb.Port(anchor=self[0]), label='down')
        self['down'].translate([0, 0.154/2, 0])
        self['down'].rotate(np.pi, [0, 1, 0])
        self['down'].rotate(-theta, around=[1, 0, 0])


class AlkanePolymer(mb.Compound):
    def __init__(self, chain_length=1):
        super(AlkanePolymer, self).__init__()
        last_monomer = CH2()
        self.add(last_monomer)
        for i in range (chain_length-1):
            current_monomer = CH2()

            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer

In [8]:
polymer = AlkanePolymer(chain_length=10)
polymer.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
# 3. 使用mBuild的聚合物类
# mBuild提供了一个预构建类来执行这个基本功能。
# 由于它被设计得更通用，它不仅仅把复制序列(n)作为参数(' A '表示单个单体，' AB '表示两个不同的单体)。
# 然后，它通过指定indices来删除atom/bead，从而将它们绑定在一起。

![](https://mbuild.mosdef.org/en/stable/_images/polymer_image.png)

In [13]:
# 4. 构造简单己烷
# 使用mBuild的打包聚合物构建器构建一个简单的己烷分子。
# 这是通过SMILES字符串装载甲烷分子来完成的。
# 指数是明确选择的，所以分子沿着正确的方向构建，不会重叠。

import mbuild as mb
from mbuild.lib.recipes.polymer import Polymer

comp = mb.load('C', smiles=True) # mBuild compound of the monomer unit
chain = Polymer()

chain.add_monomer(compound=comp,
                  indices=[1, -2],
                  separation=.15,
                  replace=True)

chain.build(n=6, sequence='A')
chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
# 5. 使用多个单体和Capping聚合物的末端
# 本例使用甲基醚和甲烷单体构建聚合物，并在其上capping上氟化和醇端基。
# 这些单体以“AB”顺序组合两次(n=2)，这意味着聚合物将包含每个单体2个(ABAB)。

# 末端基团通过add_end_groups属性添加，指定要使用的原子(index)、键的距离(separation)、每个末端基团的位置(label)，
# 以及末端基团是否复制到聚合物的头部(duplicate)。
# indices是明确选择的，所以分子沿着正确的方向构建，不会重叠。
from mbuild.lib.recipes.polymer import Polymer
import mbuild as mb

comp_1 = mb.load('C', smiles=True)
comp_2 = mb.load('COC', smiles=True)
chain = Polymer()
chain.add_monomer(compound=comp_1,
                  indices=[1, -1],
                  separation=.15,
                  replace=True)

chain.add_monomer(compound=comp_2,
                  indices=[3, -1],
                  separation=.15,
                  replace=True)


chain.add_end_groups(mb.load('O',smiles=True), # Capping off this polymer with an Alcohol
                     index=1,
                     separation=0.15, label="head", duplicate=False)

chain.add_end_groups(mb.load('F',smiles=True), # Capping off this polymer with a Fluorine
                     index=1,
                     separation=0.18, label="tail", duplicate=False)


chain.build(n=2, sequence='AB')
chain.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
# 6. 构建烷烃体系
# 可以通过简单地克隆上述构建的聚合物并在空间中平移和/或旋转烷烃来构建烷烃体系。
# mBuild提供了许多例程，可用于创建不同的模式，聚合物可以转移到这些模式。
comp = mb.load('C', smiles=True) # mBuild compound of the monomer unit
polymer = Polymer()

polymer.add_monomer(compound=comp,
                    indices=[1, -2],
                    separation=.15,
                    replace=True)

polymer.build(n=10, sequence='A')
polymer.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
# 我们生成的图案在xy平面上放置点，所以我们将旋转聚合物,使它垂直于xy平面
polymer.rotate(np.pi/2, [1, 0, 0])
polymer.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [21]:
# 定义一个包含所有聚合物的化合物
system = mb.Compound()

# 创建一个点的图案来填充磁盘
# 模式在0到1之间生成，因此需要缩放以提供适当的间距
pattern_disk = mb.DiskPattern(50)
pattern_disk.scale(5)
# 现在克隆聚合物并将其移动到图案中的点上

for pos in pattern_disk:
    current_polymer = mb.clone(polymer)
    current_polymer.translate(pos)
    system.add(current_polymer)

system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
# 可以使用其他模式，例如Grid3DPattern。我们也可以使用旋转命令来随机化方向。

import random

comp = mb.load('C', smiles=True)
polymer = Polymer()

polymer.add_monomer(compound=comp,
                    indices=[1, -2],
                    separation=.15,
                    replace=True)

polymer.build(n=10, sequence='A')

system = mb.Compound()
polymer.rotate(np.pi/2, [1, 0, 0])

pattern_disk = mb.Grid3DPattern(5, 5, 5)
pattern_disk.scale(8.0)

for pos in pattern_disk:
    current_polymer = mb.clone(polymer)
    for around in [(1, 0, 0), (0, 1, 0), (0, 0, 1)]:  # rotate around x, y, and z
        current_polymer.rotate(random.uniform(0, np.pi), around)
    current_polymer.translate(pos)
    system.add(current_polymer)

system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [23]:
# mBuild还提供了一个到PACKMOL的接口，允许创建随机配置。
comp = mb.load('C', smiles=True) # mBuild compound of the monomer unit
polymer = Polymer()

polymer.add_monomer(compound=comp,
                    indices=[1, -2],
                    separation=.15,
                    replace=True)

polymer.build(n=5, sequence='A')

system = mb.fill_box(polymer, n_compounds=100, overlap=1.5, box=[10,10,10])
system.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [24]:
# 7. Variations
# 与线性链不同，我们编写的聚合物类可以很容易地改变，这样每个端口都有小的扰动。
# 为了避免从平衡角度积累偏差，我们将每次克隆一个未受干扰的单体(即，monomer_proto)，然后再应用随机变化。
# 我们还定义了一个变量delta，它将控制扰动的最大量。
# 注意，较大的delta值可能会导致链本身重叠，因为mBuild目前不包含排除这种重叠的例程。

In [25]:
import mbuild as mb

import random

class AlkanePolymer(mb.Compound):
    def __init__(self, chain_length=1, delta=0):
        super(AlkanePolymer, self).__init__()
        monomer_proto = CH2()
        last_monomer = CH2()
        last_monomer['down'].rotate(random.uniform(-delta,delta), [1, 0, 0])
        last_monomer['down'].rotate(random.uniform(-delta,delta), [0, 1, 0])
        self.add(last_monomer)
        for i in range(chain_length-1):
            current_monomer = mb.clone(monomer_proto)
            current_monomer['down'].rotate(random.uniform(-delta,delta), [1, 0, 0])
            current_monomer['down'].rotate(random.uniform(-delta,delta), [0, 1, 0])
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer

polymer = AlkanePolymer(chain_length = 200, delta=0.4)
polymer.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol